In [10]:
import streamlit as st
from langchain_community.llms import Ollama
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
import os

@st.cache_resource
def setup_qa_chain():
    # Load and embed documents
    from langchain.document_loaders import TextLoader
from pathlib import Path

def load_utf8_safe(path):
    try:
        return TextLoader(path, encoding="utf-8").load()
    except UnicodeDecodeError:
        return TextLoader(path, encoding="cp1252").load()  # Windows fallback

docs = []
for filepath in Path("documents").rglob("*.txt"):
    docs.extend(load_utf8_safe(str(filepath)))

    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(docs)

    embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectordb = FAISS.from_documents(chunks, embedding)

    llm = Ollama(model="llama3")

    qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectordb.as_retriever())
    return qa

st.set_page_config(page_title="Local RAG Chat", page_icon="💬")
st.title("🧠 RAG Chat with LLaMA 3")

qa = setup_qa_chain()

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

user_input = st.chat_input("Ask something about your documents...")

if user_input:
    with st.spinner("Thinking..."):
        result = qa.invoke(user_input)

    st.session_state.chat_history.append(("user", user_input))
    st.session_state.chat_history.append(("bot", result["result"]))

# Display chat history
for role, text in st.session_state.chat_history:
    if role == "user":
        st.chat_message("user").markdown(text)
    else:
        st.chat_message("assistant").markdown(text)


SyntaxError: 'return' outside function (3829690924.py, line 37)

In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(chunks, embedding)


C:\Users\mfairbourne\AppData\Local\Temp\ipykernel_23312\1408457153.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
llm = Ollama(model="llama3")
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectordb.as_retriever())


In [ ]:
question = "What is this document about?"
response = qa.invoke(question)
print(response["result"])


In [ ]:
from IPython.display import display, Markdown

while True:
    q = input("\nAsk a question (or 'exit'): ")
    if q.lower() == "exit":
        break
    a = qa.invoke(q)["result"]
    display(Markdown(f"**Answer:** {a}"))
